In [ ]:
from zipList import N_zips # Zip codes selected
from zipList import N_name # Neighborhood names of zip codes selected

import json
import pandas as pd
import numpy as np
import requests
import ast
from config import api_key1 # Yelp API key

In [14]:
def search(offset_num,loc): 

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {'Authorization': 'Bearer {}'.format(api_key1)}

    url_params = {
    # Fast food, grocery stores, wine bars, hotels, and laundromats
    'categories': 'hotdogs,grocery,wine_bars,hotels,laundromat',
    'location': loc.replace(' ', '+'),
    'state': "New York",
    'offset': offset_num,
    'limit':50 # Maximum return of results per request
    }

    response = requests.get(url, headers=headers, params=url_params)

    return response.json() # Returns a JSON.

In [ ]:
# Initialize df schema (headers, no rows)
cols = pd.DataFrame.from_dict(search(0,'10012')['businesses']).columns
df = pd.DataFrame(columns = cols)
df['api_zip'] = None

df.head()

In [153]:
# List of lists to list 
print("Before: ", N_zips[:2])
all_zips = []
for n, i in enumerate(N_zips):
    for j in N_zips[n]:
        all_zips.append(j)
print("After: ", all_zips[:5])

Before:  [['11207', '11208'], ['11206', '11221', '11237']]
After:  ['11207', '11208', '11206', '11221', '11237']


In [137]:
# Runs for 20-30 minutes
# For each zip, for each offset 0-1000
for zipcode in all_zips:
    print(zipcode)
    for offset in range(0,1000,50):
        print(offset)
        df_append = pd.DataFrame.from_dict(search(offset, zipcode)['businesses'])
        df_append['api_zip'] = zipcode
        df = df.append(df_append)

11207
0


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11208
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11206
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11221
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11237
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
10029
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
10035
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11203
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11210
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11225
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11226
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11204
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
11235
0
50
1

In [138]:
df = df.drop_duplicates('id')
len(df)

6171

In [227]:
# Forgot to clear early test df
df_results = df.copy()

In [228]:
df_results['actual_business_zip'] = df_results['location'].apply(lambda i: i['zip_code'])

In [156]:
nabe_zip_dict = dict(zip(N_names,N_zips))

In [175]:
def get_nabe_from_zip(zipcode):
    for k, v in nabe_zip_dict.items():
        if str(zipcode) in v:
            return k

In [229]:
df_results['actual_business_neighborhood'] = df_results['actual_business_zip'].apply(get_nabe_from_zip)

In [230]:
df_results.head()

,actual_business_zip,alias,api_zip,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,actual_business_neighborhood
0,11221,l-antagoniste-bedford-stuyvesant,11207,"[{'alias': 'french', 'title': 'French'}, {'ali...","{'latitude': 40.685019, 'longitude': -73.929617}",(917) 966-5300,3278.805500,mxDfu4bHwSXoCr3FRpyMsw,https://s3-media1.fl.yelpcdn.com/bphoto/plBppj...,False,"{'address1': '238 Malcolm X Blvd', 'address2':...",L'Antagoniste,+19179665300,$$$,4.5,238,[],https://www.yelp.com/biz/l-antagoniste-bedford...,Williamsburg
1,11385,julias-ridgewood,11207,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...","{'latitude': 40.703143, 'longitude': -73.902956}",(917) 909-1314,3628.844325,9kCcNKe7z9GZFU3NhkoExA,https://s3-media3.fl.yelpcdn.com/bphoto/qx50a0...,False,"{'address1': '818 Woodward Ave', 'address2': '...",Julia's,+19179091314,$$,4.5,179,[],https://www.yelp.com/biz/julias-ridgewood?adju...,None
2,11233,peri-peri-grill-house-brooklyn,11207,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...","{'latitude': 40.68474, 'longitude': -73.92916}",(917) 966-8614,3226.910705,KPGZ6zr6lUX9YROjGcUbtw,https://s3-media3.fl.yelpcdn.com/bphoto/Pua9bm...,False,"{'address1': '235 Malcolm X Blvd', 'address2':...",Peri Peri Grill House,+19179668614,$,4.5,74,[],https://www.yelp.com/biz/peri-peri-grill-house...,None
3,11207,the-evergreen-brooklyn,11207,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...","{'latitude': 40.686017, 'longitude': -73.908236}",(718) 484-7399,1986.507300,HsFv6pTnpIGya6Yf5ztn_g,https://s3-media3.fl.yelpcdn.com/bphoto/-5Qovb...,False,"{'address1': '109 Moffat St', 'address2': None...",The Evergreen,+17184847399,$,4.5,42,[],https://www.yelp.com/biz/the-evergreen-brookly...,East New York
4,11206,mr-kiwis-brooklyn,11207,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...","{'latitude': 40.696916359286, 'longitude': -73...",(718) 453-2640,4396.977388,uaz8YKPbscDa7krfXvK_tw,https://s3-media3.fl.yelpcdn.com/bphoto/TUw8oo...,False,"{'address1': '957 Broadway', 'address2': '', '...",Mr Kiwi's,+17184532640,$$,4.5,161,[],https://www.yelp.com/biz/mr-kiwis-brooklyn?adj...,Williamsburg


In [231]:
df_results_filtered = df_results[~df_results['actual_business_neighborhood'].isnull()]

In [234]:
df_results_filtered[df_results_filtered['actual_business_neighborhood'] == 'East New York']

,actual_business_zip,alias,api_zip,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,actual_business_neighborhood
3,11207,the-evergreen-brooklyn,11207,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...","{'latitude': 40.686017, 'longitude': -73.908236}",(718) 484-7399,1986.507300,HsFv6pTnpIGya6Yf5ztn_g,https://s3-media3.fl.yelpcdn.com/bphoto/-5Qovb...,False,"{'address1': '109 Moffat St', 'address2': None...",The Evergreen,+17184847399,$,4.5,42,[],https://www.yelp.com/biz/the-evergreen-brookly...,East New York
19,11207,stop-one-supermarket-bushwick,11207,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...","{'latitude': 40.68925, 'longitude': -73.90609}",(718) 455-2336,2221.282701,VMpCbOPly-TY63X72vDM4g,https://s3-media4.fl.yelpcdn.com/bphoto/8Ake3n...,False,"{'address1': '624 Wilson Ave', 'address2': '',...",Stop One Supermarket,+17184552336,$$,4.5,20,[],https://www.yelp.com/biz/stop-one-supermarket-...,East New York
32,11207,rico-chimi-brooklyn-2,11207,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 40.677570583568, 'longitude': -73...",(347) 715-7272,1078.197049,dKXq_7HzPjnLzZi68bBmgw,https://s3-media3.fl.yelpcdn.com/bphoto/Nym74F...,False,"{'address1': '2928 Atlantic Ave', 'address2': ...",Rico Chimi,+13477157272,$,3.0,31,"[pickup, delivery]",https://www.yelp.com/biz/rico-chimi-brooklyn-2...,East New York
43,11207,zun-laundromat-bushwick,11207,"[{'alias': 'laundromat', 'title': 'Laundromat'}]","{'latitude': 40.686882314401, 'longitude': -73...",(347) 822-9231,2311.278078,WnldaMKRLpD-WOnMwjDrYw,https://s3-media1.fl.yelpcdn.com/bphoto/H4IWtJ...,False,"{'address1': '1287 Bushwick Ave', 'address2': ...",Zun Laundromat,+13478229231,$,4.5,11,[],https://www.yelp.com/biz/zun-laundromat-bushwi...,East New York
7,11208,hotel-bliss-new-york,11207,"[{'alias': 'hotels', 'title': 'Hotels'}]","{'latitude': 40.6640522181988, 'longitude': -7...",(718) 272-5564,1716.792409,sY7loHhOtarkEVk_TKb55A,https://s3-media3.fl.yelpcdn.com/bphoto/2InU1H...,False,"{'address1': '2330 Linden Blvd', 'address2': '...",Hotel Bliss,+17182725564,$,3.5,10,[],https://www.yelp.com/biz/hotel-bliss-new-york?...,East New York
9,11208,long-cheng-brooklyn,11207,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...","{'latitude': 40.68004, 'longitude': -73.88503}",(718) 235-4829,1332.825615,31fcTAZjm1MG7abejOiI-Q,https://s3-media1.fl.yelpcdn.com/bphoto/b-2AuG...,False,"{'address1': '2977 Fulton St', 'address2': '',...",Long Cheng,+17182354829,$,4.5,4,[],https://www.yelp.com/biz/long-cheng-brooklyn?a...,East New York
17,11207,ecy-market-brooklyn,11207,"[{'alias': 'grocery', 'title': 'Grocery'}]","{'latitude': 40.682144, 'longitude': -73.909618}",(718) 602-3000,1723.799885,ZguhzcVYXnKU-VRSab1YIQ,,False,"{'address1': '1788 Broadway', 'address2': '', ...",ECY Market,+17186023000,$$,5.0,3,[],https://www.yelp.com/biz/ecy-market-brooklyn?a...,East New York
19,11207,farm-country-outlet-brooklyn,11207,"[{'alias': 'grocery', 'title': 'Grocery'}]","{'latitude': 40.6628881, 'longitude': -73.8812...",(718) 272-1405,1484.278674,xc9OOVKQVR4Sgbcq-S8Jhw,https://s3-media4.fl.yelpcdn.com/bphoto/I1tEKy...,False,"{'address1': '2205 Linden Blvd', 'address2': '...",Farm Country Outlet,+17182721405,$$,3.0,11,[],https://www.yelp.com/biz/farm-country-outlet-b...,East New York
20,11208,white-castle-brooklyn-10,11207,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...","{'latitude': 40.67926, 'longitude': -73.88113}",(718) 827-6057,1505.835233,HSQodPxknZmHsiKQOqtQ5A,https://s3-media1.fl.yelpcdn.com/bphoto/-A6WeY...,False,"{'address1': '3101 Atlantic Ave', 'address2': ...",White Castle,+17188276057,$,3.0,13,[],https://www.yelp.com/biz/white-castle-brooklyn...,East New York
22,11207,popeyes-louisiana-kitchen-brooklyn-34,11207,"[{'alias': 'chicken_wings', 'title': 'Chicken ...","{'latitude': 40.680348, 'longitude': -73.90271}",(347) 529-0375,1208.517012,XWyu45YKfY-L3emB_dOJrw,https://s3-media2.fl.yelpcdn.co

In [246]:
df_results_filtered['search_category'] = ['Fast Food' if 'hotdogs' in str(i) 
 else 'Laundromat' if 'laundromat' in str(i)
 else 'Grocery Store' if 'grocery' in str(i)
 else 'Wine Bar' if 'wine_bars' in str(i)
 else 'Hotel' if 'hotel' in str(i)
 else 'Other' for i in df_results_filtered['categories']]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [249]:
df_results_filtered.to_excel('yelp_data.xlsx')

In [235]:
len(df_results_filtered)

2503

In [233]:
df_results_filtered['actual_business_neighborhood'].value_counts()

Chelsea                   326
Williamsburg              174
East Flatbush             169
East Village              151
Flatiron District         127
Tribeca                   114
East New York              89
Murray Hill                86
East Harlem                85
West Village               72
Tudor City                 68
Parkchester                66
Turtle Bay                 63
West Brighton              60
Jamaica Estates            49
Heartland Village          47
Bath Beach                 47
SoHo                       46
Ozone Park                 40
Fort Greene                40
Mill Basin                 39
Bayside                    38
Bensonhurst                38
Fresh Meadows              38
Gravesend                  34
Midwood                    34
Canarsie                   32
Maspeth                    27
Sheepshead Bay             27
Auburndale                 26
Emerson Hill               25
South Ozone Park           25
Dyker Heights              21
Rossville 